In [149]:
import pandas as pd
import numpy as np
wine=pd.read_csv('winequality-red.csv',sep=";")

In [150]:
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [151]:
X=wine.loc[:,['density']].values
t=wine['alcohol'].values

In [152]:
from sklearn.model_selection import train_test_split

In [153]:
X_train,X_test,t_train,t_test=train_test_split(X,t,test_size=0.1,random_state=0)

In [154]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X_train_std = standardScaler.fit_transform(X_train)

In [155]:
from sklearn.linear_model import LinearRegression

In [156]:
slr=LinearRegression()
slr.fit(X_train_std,t_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [157]:
X_train_std_a=np.column_stack((np.ones(X_train_std.shape[0]),X_train_std))

In [158]:
X_test_std = standardScaler.transform(X_test)
X_test_std_a = np.column_stack((np.ones(X_test_std.shape[0]),X_test_std))

In [159]:
w=np.array([slr.intercept_,slr.coef_])
w

array([10.416504517025738, array([-0.53397767])], dtype=object)

# 課題４
(発展) 数値微分と勾配法によってモデルパラメータを求めよ。（標準化して実装する）

In [160]:
def numerical_gradient(f,x):
    h=1.0e4
    grad=np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val=x[idx]
        x[idx]=tmp_val+h
        fxh1=f(x)
        
        x[idx]=tmp_val-h
        fxh2=f(x)
        
        grad[idx]=(fxh1-fxh2)/(2*h)
        
        x[idx]=tmp_val
        
    return grad

In [161]:
class DG:
    def __init__(self,X):
        self.val={}
        self.val['w']=np.random.rand(X.shape[1])
        #ランダムに重み係数を作成
        
    def predict(self,X):
        w=self.val['w']
        y=w.dot(X.T)
        return y
    
    def loss(self,X,t):#損失関数の値を返す
        y=self.predict(X)
        
        E=1/2*(y-t).dot((y-t).T)
        return E
    
    def gradiented(self,X,t):#数値微分を行う
        loss_w=lambda w:self.loss(X,t)
        
        grads={}
        grads['w']=numerical_gradient(loss_w,self.val['w'])
        
        return grads

In [162]:
lr=0.00001#学習係数の決定
loss_values=[]#損失関数の値を入れるためのからのリストを作成

In [163]:
Gradient=DG(X_train_std_a)

In [164]:
for i in range(10000):
    grad=Gradient.gradiented(X_train_std_a,t_train)
    for key in ('w'):
        Gradient.val[key]-=lr*grad[key]#勾配法により、更新

    loss=Gradient.loss(X_train_std_a,t_train)
    loss_values.append(loss)#loss_valuesというリストに損失関数の値を追加していく

In [165]:
loss_values[-1]#loss_valuesの最後の値を表示

615.5460062629932

In [166]:
Gradient.val#最適化された重み係数の表示

{'w': array([10.41650452, -0.53397767])}

In [167]:
Gradient.loss(X_test_std_a,t_test)#テストデータを入れた時の訓練データの値

68.47656628265597

In [168]:
def numerical_gradient(f,x):
    h=1.0e4
    grad=np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val=x[idx]
        x[idx]=tmp_val+h
        fxh1=f(x)
        
        x[idx]=tmp_val-h
        fxh2=f(x)
        
        grad[idx]=(fxh1-fxh2)/(2*h)
        
        x[idx]=tmp_val
        
    return grad

In [169]:
def f(w):
    X=X_train_std_a
    t=t_train
    
    y=w.dot(X.T)
    E=1/2*(y-t).dot((y-t).T)

    return E

In [170]:
w3=0.01*np.random.rand(X_train_std_a.shape[1])

lr=0.001

In [171]:
w3

array([0.00961591, 0.00457848])

In [172]:
for i in range(1000):
    w3-=lr*numerical_gradient(f,w3)
    
    

In [173]:
w3

array([10.41650452, -0.53397767])

In [188]:
def loss_2(X,t):
    
    y=w3.dot(X.T)
    E=1/2*(y-t).dot((y-t).T)
    return E

In [189]:
loss_2(X_test_std_a,t_test)

68.47656628265162

(160,)